In [1]:
import numpy as np
import pandas as pd
import os
import glob
import cv2
import random
from PIL import Image
import tensorflow as tf




In [2]:


############################################


def get_label_color_values(label_folder_path):
    label_images_names = os.listdir(label_folder_path)
    rgb_values = get_pixel_RGB_values(label_folder_path + '/' + label_images_names[0])
    
    return rgb_values



  
############################################


def get_pixel_RGB_values(path):
    im = Image.open(path)
    rgb_values = []
    pix = im.load()
    
    return list(set(list(im.getdata())))




  
############################################


def label_image_to_one_hot(image, label_values):
    semantic_map = []
    for colour in label_values:
        # colour_map = np.full((label.shape[0], label.shape[1], label.shape[2]), colour, dtype=int)
        equality = np.equal(image, colour)
        class_map = np.all(equality, axis = -1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1)

    return semantic_map



############################################


def get_input_batches_(images_path, labels_path):
    
    def generate_batches(batch_size, label_values):
        count = 0
        images_names = os.listdir(images_path)
        total_no_of_images = len(images_names)
        background = np.array([255, 0, 0])
        random.shuffle(images_names)
        for index in range(0, total_no_of_images, batch_size):
            images_arr = []
            labels_arr = []
            for image_name in images_names[index : index + batch_size]:
                name_split = image_name.split('_')
                label_name = name_split[0] + '_road_' + name_split[1]
                
                image = cv2.resize(cv2.imread(images_path + "/" + image_name), (612, 184))
#                 image = image / 255.0
                
                label = cv2.resize(cv2.imread(labels_path + "/" + label_name), (612, 184))
#                 label = label_image_to_one_hot(label, label_values)
                gt_bg = np.all(label == background, axis=2)
                gt_bg = gt_bg.reshape(*gt_bg.shape, 1)
                gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)

        
     
                
                images_arr.append(image)
                labels_arr.append(gt_image)
#                 print(np.shape(image), np.shape(label))
            yield np.array(images_arr), np.array(labels_arr)
    
    return generate_batches
  
  
    

In [4]:
label_values = get_label_color_values('/Users/vijay/Downloads/Datasets/data_road/training/gt_image_2/')

get_batches_ = get_input_batches_('/Users/vijay/Downloads/Datasets/data_road/training/image_2/', 
                                 '/Users/vijay/Downloads/Datasets/data_road/training/gt_image_2/')

In [5]:

VGG_16_weights_dict = np.load('/Users/vijay/Downloads/vgg16_weights.npz')
num_of_classes = len(label_values)

############################################

def get_placeholders_for_train_label_data(no_of_classes):
    
    x       = tf.placeholder(tf.float32, shape = [None, 224 * 224 * 3], name = 'x_rs')
    x_rs    = tf.reshape(x, shape = [-1, 224, 224, 3])
    
    y       = tf.placeholder(tf.float32, shape = [None, no_of_classes])
    y_rs    = tf.reshape(y, shape = [-1, no_of_classes])
    dropout = tf.placeholder(tf.float32, name = 'dropout')
    
    return x_rs, y_rs, dropout


############################################

def get_scale_and_gamma_variables(shape):

    scale = tf.Variable(tf.ones(shape))
    offset = tf.Variable(tf.zeros(shape))
    
    return scale, offset



############################################

def batch_normalize(data, shape):

    scale, offset = get_scale_and_gamma_variables(shape)
    mean, var     = tf.nn.moments(data, axes = [0, 1, 2])
    data_norm     = tf.nn.batch_normalization(data, mean, var, scale, offset, 0.05, name = "BN")
    
    return data_norm
  
  
  

############################################

def get_weights_or_bias_from_VGG(wts_or_bias_name):
    return VGG_16_weights_dict[wts_or_bias_name]




############################################

def get_weights_or_bias_as_variables(wts_or_bias_name):
   
    wts_or_bias    = get_weights_or_bias_from_VGG(wts_or_bias_name)
    wt_or_bias_init = tf.constant_initializer(value = wts_or_bias, dtype = tf.float32)
    wt_or_bias_var  = tf.get_variable(shape = np.shape(wts_or_bias), initializer = wt_or_bias_init, 
                                                                                  name = wts_or_bias_name)

    return wt_or_bias_var



############################################

def fetch_and_reshape_fc_weights(fc_wts_name, rs_shape):
    fc_wts      = VGG_16_weights_dict[fc_wts_name]
    fc_wts      = fc_wts.reshape(rs_shape)
    fc_wts_init = tf.constant_initializer(value = fc_wts, dtype = tf.float32)
    fc_wts_var  = tf.get_variable(shape = rs_shape, initializer = fc_wts_init, name = 'fc_wts')
    
    return fc_wts_var



############################################

def conv_layer(data, flt_name, bias_name, name):

    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        conv_wt       = get_weights_or_bias_as_variables(flt_name)
        conv_bias     = get_weights_or_bias_as_variables(bias_name)

        conv_res      = tf.nn.conv2d(data, filter = conv_wt, strides = [1, 1, 1, 1], padding = 'SAME')
        conv_res_bias = tf.nn.bias_add(conv_res, conv_bias)
        conv_bn       = batch_normalize(conv_res_bias, conv_res_bias.get_shape().as_list()[-1])
        conv_res_relu = tf.nn.relu(conv_bn)

        return conv_res_relu



############################################

def pooling_layer(data, pool_name):
    with tf.variable_scope(pool_name, reuse = tf.AUTO_REUSE):
        return tf.nn.max_pool(value = data, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

    

############################################

def fc_layer(data, dropout, fc_wts_name = None, fc_bias_name = None):
    
    with tf.variable_scope(fc_wts_name, reuse = tf.AUTO_REUSE):
        
        
        if fc_wts_name != 'fc8_W':
            fc_wt_shape = []
            if fc_wts_name == 'fc6_W': 
                fc_wt_shape      = [7, 7, 512, 4096]
                
            if fc_wts_name == 'fc7_W':
                fc_wt_shape      = [1, 1, 4096, 4096]

            fc_wts       = fetch_and_reshape_fc_weights(fc_wts_name, fc_wt_shape)            
            fc_bias      = get_weights_or_bias_as_variables(fc_bias_name)

            fc_conv      = tf.nn.conv2d(data, filter = fc_wts, strides = [1, 1, 1, 1], padding = 'SAME')
            fc_conv_bias = tf.nn.bias_add(fc_conv, fc_bias)
            fc_conv_relu = tf.nn.relu(fc_conv_bias)
            return tf.nn.dropout(fc_conv_relu, rate = dropout)

        elif fc_wts_name == 'fc8_W':
            initial      = tf.truncated_normal([1, 1, 4096, num_of_classes], stddev=0.0001)
            kernel       = tf.get_variable('kernel', initializer=initial)
            conv         = tf.nn.conv2d(data, kernel, [1, 1, 1, 1], padding='SAME')
            bias         = tf.Variable(tf.constant(0.1, shape = [num_of_classes]), name = 'bias')
            return tf.nn.bias_add(conv, bias)
        
        
        
        

        
############################################

def skip_connection(from_data, con_name, flt_val, num_classes):
    with tf.variable_scope(con_name, reuse=tf.AUTO_REUSE):
        skp_wt = tf.Variable(tf.truncated_normal(flt_val, stddev = 0.2, seed = 23, dtype = tf.float32), name = 'skip_wt')
        skp_conv = tf.nn.conv2d(from_data, skp_wt, [1, 1, 1, 1], padding = 'SAME')
        
        skp_bias = tf.Variable(tf.constant(0.1, shape = [num_classes], dtype = tf.float32), name = 'skip_bias')
        skip_res = tf.nn.bias_add(skp_conv, skp_bias)
        return skip_res

    

############################################

def upsample(from_data, op_shape, no_of_channels, name, factor, num_classes):
    flt_size = 2 * factor - factor % 2
    strides = [1, factor, factor, 1]
    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        filter_shape = [flt_size, flt_size, no_of_channels, no_of_channels]
        weights = get_bilinear_weights(filter_shape, factor)
        deconv = tf.nn.conv2d_transpose(from_data, weights, op_shape, strides = strides, padding = 'SAME')
        
        bias_init = tf.constant_initializer(0.1)
        
        upsmp_bias = tf.get_variable(initializer = bias_init, shape = [num_classes], name = 'upsmp_bias')
        
        deconv_bias = tf.nn.bias_add(deconv, upsmp_bias)
        
    return deconv_bias




############################################

def get_bilinear_weights(filter_shape, factor):
#     print('filter_shape is ' + str(filter_shape))
    flt_size = filter_shape[1]
    if flt_size % 2 == 0:
        centre = factor - 1
    else:
        centre = factor - 0.5
        
    bilinear = np.zeros([filter_shape[0], filter_shape[1]])
    weights = np.zeros(filter_shape)
#     print('weights shape is ' + str(np.shape(weights)))
    for row in range(filter_shape[0]):
        for column in range(filter_shape[1]):
            cell_val = (1 - abs((row - centre) / factor)) * (1 - abs((column - centre) / factor))
            bilinear[row, column] = cell_val
    
    for val in range(filter_shape[2]):
        weights[:, :, val, val] = bilinear
    
    wt_init = tf.constant_initializer(value = weights, dtype = tf.float32)
    bilinear_weights = tf.get_variable(initializer = wt_init, name = 'blnr_wt', shape = filter_shape)
    
    return bilinear_weights




(101, 720, 960, 3)

In [7]:


batch_size = 8

x           = tf.placeholder(tf.float32, shape = [None, 184, 612, 3], name = 'x_ph')
y           = tf.placeholder(tf.float32, shape = [None, 184, 612, num_of_classes], name = 'y_ph')
dropout     = tf.placeholder(tf.float32, name = 'dropout')
global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

##########################################################
        
def FCN():
  
  
        
    with tf.variable_scope('FCN', reuse = tf.AUTO_REUSE):


        conv_1_1 = conv_layer(x, flt_name = 'conv1_1_W', bias_name = 'conv1_1_b', name = 'conv_1_1')
        conv_1_2 = conv_layer(conv_1_1, flt_name = 'conv1_2_W', bias_name = 'conv1_2_b',  name = 'conv_1_2')
        pool_1   = pooling_layer(conv_1_2, pool_name = 'pool_1')


        conv_2_1 = conv_layer(pool_1, flt_name = 'conv2_1_W', bias_name = 'conv2_1_b', name = 'conv_2_1')
        conv_2_2 = conv_layer(conv_2_1, flt_name = 'conv2_2_W', bias_name = 'conv2_2_b', name = 'conv_2_2')
        pool_2   = pooling_layer(conv_2_2, pool_name = 'pool_2')

        conv_3_1 = conv_layer(pool_2, flt_name = 'conv3_1_W', bias_name = 'conv3_1_b', name = 'conv_3_1')
        conv_3_2 = conv_layer(conv_3_1, flt_name = 'conv3_2_W', bias_name = 'conv3_2_b', name = 'conv_3_2')
        conv_3_3 = conv_layer(conv_3_2, flt_name = 'conv3_3_W', bias_name = 'conv3_3_b', name = 'conv_3_3')
        pool_3   = pooling_layer(conv_3_3, pool_name = 'pool_3')

        conv_4_1 = conv_layer(pool_3, flt_name = 'conv4_1_W', bias_name = 'conv4_1_b', name = 'conv_4_1')
        conv_4_2 = conv_layer(conv_4_1, flt_name = 'conv4_2_W', bias_name = 'conv4_2_b', name = 'conv_4_2')
        conv_4_3 = conv_layer(conv_4_2, flt_name = 'conv4_3_W', bias_name = 'conv4_3_b', name = 'conv_4_3')
        pool_4   = pooling_layer(conv_4_3, pool_name = 'pool_4')

        conv_5_1 = conv_layer(pool_4, flt_name = 'conv5_1_W', bias_name = 'conv5_1_b', name = 'conv_5_1')
        conv_5_2 = conv_layer(conv_5_1, flt_name = 'conv5_2_W', bias_name = 'conv5_2_b', name = 'conv_5_2')
        conv_5_3 = conv_layer(conv_5_2, flt_name = 'conv5_3_W', bias_name = 'conv5_3_b', name = 'conv_5_3')
        pool_5   = pooling_layer(conv_5_3, pool_name = 'pool_5')

        fc_1     = fc_layer(pool_5, dropout, 'fc6_W', 'fc6_b')
        fc_2     = fc_layer(fc_1, dropout, 'fc7_W', 'fc7_b')

        fc_3     = fc_layer(fc_2, dropout, 'fc8_W', 'fc8_b')


        batch_shape = x.get_shape().as_list()
        upsample_op_shape    = tf.stack([batch_size, batch_shape[1], batch_shape[2], num_of_classes])
        fc3_upsample         = upsample(fc_3, upsample_op_shape, num_of_classes, 'fc3_upsmp', 32, num_of_classes)

        pool4_skip_flt_shape = tf.stack([1, 1, np.shape(pool_4)[-1], num_of_classes])
        pool3_skip_flt_shape = tf.stack([1, 1, np.shape(pool_3)[-1], num_of_classes])

        pool4_skip           = skip_connection(pool_4, 'pool4_skip', pool4_skip_flt_shape, num_of_classes)
        pool4_upsample       = upsample(pool4_skip, upsample_op_shape, num_of_classes, 'pool4_upsmp', 16, num_of_classes)

        pool3_skip           = skip_connection(pool_3, 'pool3_skip', pool3_skip_flt_shape, num_of_classes)
        pool3_upsample       = upsample(pool3_skip, upsample_op_shape, num_of_classes, 'pool3_upsmp', 8, num_of_classes)

        logits = tf.add(fc3_upsample, tf.add(2 * pool4_upsample, 4 * pool3_upsample))


    return logits






    
##########################################################
    
def logistic_loss(logits, labels, n_classes):
    
    with tf.variable_scope('logistic_loss'):
        
        reshaped_logits = tf.reshape(logits, (-1, n_classes))
        reshaped_labels = tf.reshape(labels, (-1, n_classes))
        entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=reshaped_logits,
                                                          labels=reshaped_labels)
        loss = tf.reduce_mean(entropy, name='logistic_loss')
        return loss


    
    
##########################################################


def build_loss_summary(loss):
    
    with tf.variable_scope('loss_summary'):
        tf.summary.scalar('loss', loss)
        tf.summary.histogram('h_loss', loss)
        return tf.summary.merge_all()
    
    
##########################################################


def train_optimizer():
  
  with tf.variable_scope('optim', reuse = tf.AUTO_REUSE):
    FCN_logits = FCN()
    cost = logistic_loss(logits=FCN_logits, labels=y, n_classes=num_of_classes)
    
#     cost = tf.reduce_mean(tf.square(y - FCN_logits))
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.00001).minimize(cost, global_step = global_step)
    
    return FCN_logits, cost, optimizer


  
  
  
##########################################################

def substract_the_mean(images):
    mean = np.array([123.68, 116.779, 103.939]).reshape([1, 1, 1, 3]).astype(np.float32)
    return images - mean


  
##########################################################


def run_FCN_model():
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    count = 0
    with tf.Session(config = config) as session:
        
        FCN_logits, loss, optimizer = train_optimizer()
        loss_summary = build_loss_summary(loss)
        session.run(tf.global_variables_initializer())
        saver = tf.train.Saver(keep_checkpoint_every_n_hours = 0.5)
        writer = tf.summary.FileWriter('./graphs/', session.graph)

        for epoch in range(0, 1):
            
            images, labels = next(get_batches_(batch_size, label_values))
            s_images = substract_the_mean(images)

            _, loss_, summary = session.run([optimizer, loss, loss_summary], feed_dict = {x : s_images,
                                                                                         y : labels, 
                                                                                         dropout : 0.75})

            writer.add_summary(summary, global_step = epoch)
            print(epoch, loss_ / batch_size)
            if epoch % 100 == 0:
                print('saved')
                save_path = saver.save(session, "./SS_FCN_model", epoch)

    return FCN_logits


In [8]:
FCN_logits = run_FCN_model()

Instructions for updating:
Use tf.cast instead.
0 3.0746020911465166e-06
saved


In [9]:
def get_test_batch(test_file_path):
    
    def get_batch(batch_size):
        count = 0
        images_names = os.listdir(test_file_path)
        total_no_of_images = len(images_names)
   
        for index in range(0, total_no_of_images, batch_size):
            
        
            images_arr = []
    
            for image_name in images_names[index : index + batch_size]:
                
        
                image = cv2.resize(cv2.imread(test_file_path + "/" + image_name), (612, 184))
#             image = image / 255.0
            
#                 if count < 8:
#                     plt.figure(figsize=(10, 10))
# #                     plt.imshow(image)
#                     plt.imshow(label)
#                     count = count + 1
                images_arr.append(image)
   
            yield np.asarray(images_arr)
    return get_batch


get_test_images = get_test_batch('/Users/vijay/Downloads/Datasets/data_road/testing/image_2/')



In [12]:
import PIL
from PIL import Image
import scipy.misc

batch_size = 8
no_of_test_images = 290

def reverse_one_hot_to_colors(img):
    x = np.argmax(img, axis = -1)
    return x


def color_code_segmentation(image, label_values):
    colour_codes = np.array(label_values)
    x = colour_codes[image.astype(int)]

    return x


def pred_to_image(session, image, original):
    
    count = 0
    im_softmax_ = session.run([tf.nn.softmax(image)], feed_dict = {x : original, dropout : 0.5})
    print('im_softmax ' + str(np.shape(im_softmax_)))
    for pred in im_softmax_[0]:
        
        print('pred shape ' + str(np.shape(pred[:, :, 1])))
        pred = pred[:, :, 0].reshape(184, 612)
        segmentation = (pred > 0.5).reshape(184, 612, 1)
        mask = np.dot(segmentation, np.array([[221, 28, 199, 127]]))
        mask = Image.fromarray(mask, mode="RGBA")
        print('original count ' + str(np.shape(original[count])))
        street_im = Image.fromarray(np.uint8(original[count]))
        street_im.paste(mask, box=None, mask=mask)
        cv2.imwrite(('./' + str(count) + '.png'), cv2.cvtColor(np.uint8(street_im), cv2.COLOR_RGB2BGR))
        count = count + 1

def make_predictions(dropout_rate):
    
    count = 0
    total_test_images = os.listdir('/Users/vijay/Downloads/Datasets/data_road/testing/image_2/')
    with tf.Session() as session:
        
      
        meta_graph = tf.train.import_meta_graph('./SS_FCN_model-0.meta')
        meta_graph.restore(session, tf.train.latest_checkpoint('./'))
        total_batches = int(no_of_test_images / batch_size)
        for batch_no in range(total_batches):
            
            if batch_no < 1:
                images = next(get_test_images(batch_size))
                print('before FCN_logits ' + str(FCN_logits.get_shape().as_list()))
                reshape_logits = tf.reshape(FCN_logits, (-1, num_of_classes))
                print('after FCN_logits ' + str(reshape_logits.get_shape().as_list()))
            
                im = substract_the_mean(images)
                pred_to_image(session, FCN_logits, im)

            
#             
#                 prediction = session.run([tf.nn.softmax(FCN_logits)], feed_dict = {x : im, dropout : dropout_rate})

#                 image_no = 0
#                 for pred in prediction[0]:
                

#                     output_image = reverse_one_hot_to_colors((pred))
#                     out_vis_image = color_code_segmentation(output_image, label_values)
#                     cv2.imwrite(('./' + str(count) + '.png'),cv2.cvtColor(np.uint8(out_vis_image), cv2.COLOR_RGB2BGR))


#                     count = count + 1
#                     print('saved image ' + str(count) + '.png')
            
            

In [ ]:
make_predictions(0.5)